## Cargar el documento



In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.4 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader

# Abrir el archivo PDF en modo binario
with open("/content/apuntes.pdf", "rb") as file:
    reader = PdfReader(file)
    all_text = ""
    for page in reader.pages:
        all_text += page.extract_text() + "\n"

# Mostrar un fragmento del texto extraído para asegurarnos de que funciona
print(all_text[:2000])

 
 APUNTES DE HISTORIA DE ESPAÑA    
2º BACHILLERATO  
 
 
IES SAAVEDRA FAJARDO.       CURSO 2018 -19 

TEMARIO HISTORIA DE ESPAÑA.    CURSO 2018 -19 
1. La romanización  
2. Al-Ándalus: evolución política  
3. Castilla y Aragón en la Baja Edad Media  
4. Reyes Católicos  
5. Austrias Mayores: política exterior  
6. El reformismo borbónico en el siglo XVIII  
7. Las Cortes de Cádiz. La Constitución de 1812  
8. Absolutismo frente a liberalismo. Evolución política del reinado de Fernando VII.  
9. Revolución liberal en el reinado de Isabel II. Carlismo y guerra civil. Construcción y evolución del 
Estado liberal.  
10. Transformaciones económicas del siglo XIX: las desamortizaciones.  
11. El Sex enio Revolucionario (1868 -1874)  
12. El régimen de la Restauración. Características y funcionamiento del sistema canovista.  
13. Panorama general del reinado de Alfonso XIII. Intentos de modernización: el Regeneracionismo y 
crisis socio -económicas (1902 -1923)  
14. La dictadura de Primo d

## Preparación y segmentación (chunking temático)

En esta fase se llevó a cabo la extracción automática de los contenidos del documento PDF, estructurado por temas del 1 al 18.

Una vez localizados los títulos, el texto se segmentó en bloques temáticos completos, desde el inicio de cada tema hasta el comienzo del siguiente. Cada bloque fue almacenado en un diccionario denominado **chunks_all**, donde cada entrada contiene tanto el título como el contenido íntegro del tema correspondiente.

Este tipo de segmentación temáticamente informada es fundamental para un sistema RAG (Retrieval-Augmented Generation), ya que permite una recuperación precisa del contexto relevante, favoreciendo respuestas más coherentes y específicas por parte del modelo generativo.

In [3]:
import re

# Expresión regular para detectar cualquier tema (Tema 1 hasta 18)
patron_todos_los_temas = re.compile(r"(Tema\s+(\d{1,2})[^.\n]*)", flags=re.IGNORECASE)

# Buscar todos los títulos y sus ubicaciones
matches_all = list(patron_todos_los_temas.finditer(all_text))

# Extraer todos los temas del 1 al 18
chunks_all = {}
for i, match in enumerate(matches_all):
    numero_tema = match.group(2)
    titulo_detectado = match.group(1).strip()
    inicio = match.start()
    fin = matches_all[i + 1].start() if i + 1 < len(matches_all) else len(all_text)
    texto_tema = all_text[inicio:fin].strip()
    chunks_all[numero_tema] = {
        "titulo": titulo_detectado,
        "contenido": texto_tema
    }

# Mostrar los títulos detectados y un inicio del contenido
from pprint import pprint
pprint({k: {"titulo": v["titulo"], "inicio": v["contenido"][:300]} for k, v in chunks_all.items()})

{'1': {'inicio': 'TEMA 1: LA ROMANIZACIÓN  \n'
                 'I. INTRODUCCIÓN  \n'
                 ' Se entiende por romanización  el proceso de adaptación de '
                 'los pueblos hispanos, a veces por la \n'
                 'fuerza, a las estructuras económicas, sociales, políticas y '
                 'culturales del Imperio romano. Con el \n'
                 'tiempo habría de suponer la desaparición de muchos usos y co',
       'titulo': 'TEMA 1: LA ROMANIZACIÓN'},
 '10': {'inicio': 'TEMA 10. Transformaciones económicas del siglo XIX: las '
                  'desamortizaciones.  \n'
                  ' \n'
                  'I.-INTRODUCCIÓN  \n'
                  '\n'
                  'IES SAAVEDRA FAJARDO                                   2º '
                  'BACHILLERATO                                                     '
                  'HI STORIA DE ESPAÑA  \n'
                  ' \n'
                  '95 \n'
                  ' La desamortización supon

## Almacenamiento de los fragmentos temáticos

Una vez realizada la segmentación del documento original en bloques temáticos (chunking), se procedió al almacenamiento individual de cada tema en archivos de texto independientes. Esta etapa permite estructurar el corpus de forma que sea fácilmente indexable por herramientas de recuperación semántica en arquitecturas RAG (Retrieval-Augmented Generation).

Se genera un archivo .txt por cada tema. El nombre de cada archivo incluye el número de tema y parte del título, de manera que se pueda identificar fácilmente el contenido sin necesidad de abrir el archivo.

Todos los archivos fueron guardados en un directorio específico (temas_historia), y se mantienen listos para ser utilizados como documentos fuente en el proceso de indexación, creación de embeddings y consulta por parte del modelo generativo. Esta estructura por archivo permite una mayor granularidad en la recuperación de información, lo que mejora tanto la precisión como la relevancia de las respuestas generadas por el sistema.

In [4]:
import os

# Crear una carpeta para guardar los archivos de texto
output_dir = "/content/temas_historia"
os.makedirs(output_dir, exist_ok=True)

# Guardar cada tema como archivo .txt con su número y título
for numero, datos in chunks_all.items():
    # Crear un nombre de archivo con número de tema y título resumido
    titulo_limpio = re.sub(r'[^\w\s-]', '', datos["titulo"]).strip().replace(" ", "_")[:50]
    filename = f"Tema_{numero}_{titulo_limpio}.txt"
    filepath = os.path.join(output_dir, filename)

    # Escribir el contenido en el archivo
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(f"{datos['titulo']}\n\n{datos['contenido']}")

# Confirmar los nombres de archivos creados
os.listdir(output_dir)

['Tema_15_Tema_15_La_Segunda_República_1931_-36.txt',
 'Tema_1_TEMA_1_LA_ROMANIZACIÓN.txt',
 'Tema_16_Tema_16.txt',
 'Tema_14_Tema_14__La_dictadura_de_Primo_de_Rivera.txt',
 'Tema_13_Tema_13_Panorama_general_del_reinado_d_e_Alfonso_X.txt',
 'Tema_12_Tema__12.txt',
 'Tema_10_TEMA_10.txt',
 'Tema_11_TEMA_11__El_Sexenio_Revolucionario_1868_-1874.txt',
 'Tema_9_TEMA_9_REVOLUCIÓN_LIBERAL_EN_EL_REINADO_DE_ISABEL_.txt',
 'Tema_8_Tema_8_Absolutismo_frente_a_liberalismo.txt',
 'Tema_18_Tema_18.txt',
 'Tema_5_TEMA_5.txt',
 'Tema_3_TEMA_3_Castilla_y_Aragón_en_la_Baja_Edad_Media.txt',
 'Tema_7_TEMA_7.txt',
 'Tema_17_TEMA_17.txt',
 'Tema_2_TEMA_2_Al_-Ándalus_evolución_política.txt',
 'Tema_4_Tema_4.txt']

In [5]:
# Renombrar manualmente los archivos .txt:
import os

# Ruta donde están los archivos
folder = "/content/temas_historia"

# Diccionario con los nombres actuales y los nuevos que quieres poner
renombrar = {
    "Tema_1_TEMA_1_LA_ROMANIZACIÓN.txt": "Tema_1_La_Romanización.txt",
    "Tema_2_TEMA_2_Al_-Ándalus_evolución_política.txt": "Tema_2_Al_Andalus_evolución_política.txt",
    "Tema_3_TEMA_3_Castilla_y_Aragón_en_la_Baja_Edad_Media.txt": "Tema_3_Castilla_y_Aragon_Baja_Edad_Media.txt",
    "Tema_4_Tema_4.txt": "Tema_4_Los_Reyes_Católicos.txt",
    "Tema_5_TEMA_5.txt": "Tema_5_Austrias_Mayores_politica_exterior.txt",
    "Tema_7_TEMA_7.txt": "Tema_7_Las_Cortes_de_Cadiz_Constitucion_1812.txt",
    "Tema_8_Tema_8_Absolutismo_frente_a_liberalismo.txt": "Tema_8_Absolutismo_vs_Liberalismo_Fernando_VII.txt",
    "Tema_9_TEMA_9_REVOLUCIÓN_LIBERAL_EN_EL_REINADO_DE_ISABEL_.txt": "Tema_9_Reinado_Isabel_II_Carlismo_Guerra_Civil.txt",
    "Tema_10_TEMA_10.txt": "Tema_10_Desamortizaciones_y_economia_siglo_XIX.txt",
    "Tema_11_TEMA_11__El_Sexenio_Revolucionario_1868_-1874.txt": "Tema_11_El_Sexenio_Revolucionario.txt",
    "Tema_12_Tema__12.txt": "Tema_12_Restauración_y_sistema_canovista.txt",
    "Tema_13_Tema_13_Panorama_general_del_reinado_d_e_Alfonso_X.txt": "Tema_13_Reinado_Alfonso_XIII_y_Regeneracionismo.txt",
    "Tema_14_Tema_14__La_dictadura_de_Primo_de_Rivera.txt": "Tema_14_Dictadura_Primo_de_Rivera.txt",
    "Tema_15_Tema_15_La_Segunda_República_1931_-36.txt": "Tema_15_La_Segunda_Republica.txt",
    "Tema_16_Tema_16.txt": "Tema_16_La_Guerra_Civil.txt",
    "Tema_17_TEMA_17.txt": "Tema_17_El_Estado_Franquista.txt",
    "Tema_18_Tema_18.txt": "Tema_18_La_Transicion_Democratica.txt",
}

# Aplicar renombrado
for antiguo, nuevo in renombrar.items():
    origen = os.path.join(folder, antiguo)
    destino = os.path.join(folder, nuevo)
    if os.path.exists(origen):
        os.rename(origen, destino)

# Mostrar los nuevos nombres
print("Archivos renombrados:")
print(os.listdir(folder))

Archivos renombrados:
['Tema_13_Reinado_Alfonso_XIII_y_Regeneracionismo.txt', 'Tema_2_Al_Andalus_evolución_política.txt', 'Tema_18_La_Transicion_Democratica.txt', 'Tema_15_La_Segunda_Republica.txt', 'Tema_3_Castilla_y_Aragon_Baja_Edad_Media.txt', 'Tema_1_La_Romanización.txt', 'Tema_4_Los_Reyes_Católicos.txt', 'Tema_5_Austrias_Mayores_politica_exterior.txt', 'Tema_14_Dictadura_Primo_de_Rivera.txt', 'Tema_17_El_Estado_Franquista.txt', 'Tema_9_Reinado_Isabel_II_Carlismo_Guerra_Civil.txt', 'Tema_12_Restauración_y_sistema_canovista.txt', 'Tema_7_Las_Cortes_de_Cadiz_Constitucion_1812.txt', 'Tema_8_Absolutismo_vs_Liberalismo_Fernando_VII.txt', 'Tema_10_Desamortizaciones_y_economia_siglo_XIX.txt', 'Tema_16_La_Guerra_Civil.txt', 'Tema_11_El_Sexenio_Revolucionario.txt']


In [6]:
!pip install llama-index llama-index-embeddings-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.9 MB/s eta 0:00:00


## Generación de embeddings y construcción del índice vectorial

Una vez estructurado y almacenado el corpus temático en archivos individuales, se procedió a la creación del índice semántico mediante técnicas de vectorización de texto. Este paso es fundamental en la arquitectura RAG (Retrieval-Augmented Generation), ya que permite recuperar información relevante en base al significado de una consulta, no solo por coincidencia literal de palabras clave.

Para ello se utilizó la librería LlamaIndex, que proporciona una interfaz sencilla para construir índices vectoriales a partir de documentos. Los pasos seguidos fueron los siguientes:

1. Carga del corpus: Se empleó la clase SimpleDirectoryReader para cargar automáticamente todos los archivos .txt desde el directorio que contiene los temas previamente segmentados.
2. Generación de embeddings: Cada documento fue transformado en un vector numérico usando el modelo text-embedding-3-small de OpenAI. Este modelo permite representar el contenido de manera que documentos con significados similares estén cercanos en el espacio vectorial.
3. Construcción del índice: Los vectores generados fueron almacenados en un índice semántico utilizando la clase VectorStoreIndex. Este índice permite realizar búsquedas eficientes por similitud de significado entre una consulta y los documentos.
4. Persistencia del índice: Finalmente, el índice fue guardado en disco mediante la función persist(), lo que permite reutilizarlo sin necesidad de recalcular los embeddings en futuras ejecuciones.

Gracias a esta estructura, el sistema es capaz de recuperar rápidamente los temas más relevantes para cualquier pregunta formulada por el usuario, lo que constituye la base para la interacción posterior con el modelo generativo.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "tu_clave_de_api_aqui"

In [8]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import os

# Ruta donde están los archivos con los temas limpios
carpeta = "/content/temas_historia"

# Leemos todos los archivos como documentos
documents = SimpleDirectoryReader(carpeta).load_data()

# Creamos embeddings con OpenAI (requiere API key)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Creamos el índice vectorial con esos documentos
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Guardamos el índice para poder reutilizarlo luego
index.storage_context.persist(persist_dir="/content/historia_index")

# Verificamos cuántos documentos han sido indexados
len(documents)

17

## Consulta al sistema con preguntas históricas

Aunque no se haya especificado directamente en el código, el modelo generativo utilizado por defecto en este sistema RAG es gpt-3.5-turbo, proporcionado por OpenAI a través de la API configurada en el entorno (OPENAI_API_KEY). LlamaIndex por defecto utiliza gpt-3.5-turbo de OpenAI — siempre que detecte que tienes definida una variable de entorno válida: OPENAI_API_KEY.

In [9]:
# Prompt base (instrucciones para el modelo)
prompt_base = """
Eres un profesor de Historia de España experto y riguroso.
Responde únicamente utilizando la información proporcionada en los documentos del índice,
sin recurrir a conocimiento externo o general. Si no encuentras suficiente información,
indícalo con honestidad en tu respuesta.

Redacta la respuesta en español claro y formal, con un estilo académico pero comprensible
para estudiantes de nivel universitario o de bachillerato.

Si es posible, organiza la información en párrafos, incluyendo fechas, protagonistas, causas
y consecuencias relevantes. No inventes nada ni respondas con opiniones personales.
"""

In [11]:
from llama_index.core.query_engine import RetrieverQueryEngine

# Crear un query engine a partir del índice ya cargado
query_engine = index.as_query_engine(similarity_top_k=3,  # reduce si quieres más precisión (1 o 2)
    response_mode="refine"# obliga al modelo a ceñirse a los documentos recuperados)
)

# Pregunta específica del usuario
pregunta_usuario = "¿Qué consecuencias tuvo el Sexenio Revolucionario?"

# Combinar prompt + pregunta
prompt_final = f"{prompt_base}\n\nPregunta del estudiante: {pregunta_usuario}"

# Hacer la consulta
respuesta = query_engine.query(prompt_final)

# Mostrar resultado
print(respuesta.response)

El Sexenio Revolucionario en España, que abarcó desde 1868 hasta 1874, tuvo como consecuencias relevantes la destronación de la reina Isabel II en 1868, el establecimiento de un gobierno provisional y un régimen democrático. Además, se promulgó la Constitución de 1869, que instauró una monarquía constitucional y una división de poderes.

Otra consecuencia importante fue la Guerra de los Diez Años en Cuba, que se inició en 1868 y se prolongó hasta 1878, generando impactos significativos en la economía y la estabilidad política de España, así como tensiones internas y externas.

La alternancia en el poder entre diferentes grupos políticos durante el Sexenio Revolucionario provocó inestabilidad y conflictos internos, sentando las bases para la posterior Restauración borbónica con la llegada de Alfonso XII al trono en 1874, poniendo fin a este periodo de transformaciones y convulsiones.


In [12]:
# Pregunta específica del usuario
pregunta_usuario = "Dime un resumen de la dictadura de Francisco Franco"

# Combinar prompt + pregunta
prompt_final = f"{prompt_base}\n\nPregunta del estudiante: {pregunta_usuario}"

# Hacer la consulta
respuesta = query_engine.query(prompt_final)

# Mostrar resultado
print(respuesta.response)

El régimen de Francisco Franco se estableció tras la guerra civil española en 1939 y se mantuvo hasta 1975. Durante este periodo, Franco concentró todo el poder en sus manos, estableciendo una dictadura personal similar a regímenes fascistas. Su régimen se caracterizó por un profundo antiliberalismo, represión interna contra los republicanos, y una política económica de autarquía. A nivel internacional, el franquismo se vio aislado y fue expulsado de organismos internacionales. Franco se apoyó en elementos de la Falange Española y los tradicionalistas, promoviendo una visión nacional-católica.


In [13]:
# Pregunta específica del usuario
pregunta_usuario = "¿Quién es Leo Messi?"

# Combinar prompt + pregunta
prompt_final = f"{prompt_base}\n\nPregunta del estudiante: {pregunta_usuario}"

# Hacer la consulta
respuesta = query_engine.query(prompt_final)

# Mostrar resultado
print(respuesta.response)

No se proporciona información relevante en el contexto para responder a la pregunta sobre Leo Messi.
